In [22]:
# %matplotlib widget
# %matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import pandas as pd
import pickle
import h5py
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# ferritin
# https://docs.google.com/spreadsheets/d/1sZtDsfgdbVltXtC_nw7wt7QoXf_mQ2X5-X3lCEDUEiI/edit#gid=1882865384

In [2]:
sheet_url = "https://docs.google.com/spreadsheets/d/1sZtDsfgdbVltXtC_nw7wt7QoXf_mQ2X5-X3lCEDUEiI/edit#gid=1882865384"
url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [3]:
elog = pd.read_csv(url)
elog.columns

Index(['Sample', 'comment', 'Short comment', 'Sample no. (label)',
       'measurement folder', 'scan number', 'LN flow meter', 'Temperature, K',
       'Temperature, C', 'Exposure time (s)', 'number of frames',
       'Mesh (spots x lines)', 'reps per spot', 'No of spots',
       'Total exposure time (s)', 'transmission (%)', 'Absorbers 80um',
       'Absorbers 100um', 'position y, mm', 'position z, mm'],
      dtype='object')

In [5]:
# # save elog
# elog.to_pickle("../02-source/elog")

In [2]:
# load elog as pd dataframe
elog = pd.read_pickle("../03-source/elog")

In [3]:
elog.head(5)

,Sample,comment,Short comment,Sample no. (label),measurement folder,scan number,LN flow meter,"Temperature, K","Temperature, C",Exposure time (s),number of frames,Mesh (spots x lines),reps per spot,No of spots,Total exposure time (s),transmission (%),Absorbers 80um,Absorbers 100um,"position y, mm","position z, mm"
0,silica_gly23,NaN,empty,1,silica_gly23_0001,12,NaN,NaN,NaN,0.0001,10000.0,1,NaN,NaN,1.000,NaN,0.0,0.0,NaN,NaN
1,silica_gly23,NaN,empty,1,silica_gly23_0001,13,NaN,NaN,NaN,0.0010,10000.0,1,NaN,NaN,10.000,NaN,0.0,0.0,NaN,NaN
2,silica_gly23_SAXS,NaN,empty,1,silica_gly23_saxs_0001,2,NaN,NaN,25,0.0010,1.0,1,NaN,NaN,0.001,NaN,0.0,0.0,NaN,NaN
3,silica_gly23,NaN,NaN,1,silica_gly23_saxs_0001,3,NaN,NaN,25,0.1000,1.0,1,NaN,NaN,0.100,NaN,0.0,0.0,NaN,NaN
4,silica_gly23,NaN,NaN,1,silica_gly23_saxs_0001,4,NaN,NaN,25,0.0100,1.0,1,NaN,NaN,0.010,NaN,0.0,0.0,NaN,NaN


In [8]:
cols = list(elog.columns)
cols

['Sample',
 'comment',
 'Short comment',
 'Sample no. (label)',
 'measurement folder',
 'scan number',
 'LN flow meter',
 'Temperature, K',
 'Temperature, C',
 'Exposure time (s)',
 'number of frames',
 'Mesh (spots x lines)',
 'reps per spot',
 'No of spots',
 'Total exposure time (s)',
 'transmission (%)',
 'Absorbers 80um',
 'Absorbers 100um',
 'position y, mm',
 'position z, mm']

In [10]:
condition = elog['measurement folder'].str.contains('ferritin_conc_gly_50_6_0005', na=False)
print("Temp: ", elog[condition]["Temperature, K"].values)
print("Transm: ", elog[condition]["transmission (%)"].values)

elog_entries = elog[condition]
elog_entries

Temp:  [220.]
Transm:  ['100']


,Sample,comment,Short comment,Sample no. (label),measurement folder,scan number,LN flow meter,"Temperature, K","Temperature, C",Exposure time (s),number of frames,Mesh (spots x lines),reps per spot,No of spots,Total exposure time (s),transmission (%),Absorbers 80um,Absorbers 100um,"position y, mm","position z, mm"
135,ferritin_conc_gly_50_6,"xpcs, 4 reps per spot, 220K, full beam",NaN,10.6,ferritin_conc_gly_50_6_0005,1-460,~2,220.0,NaN,0.0002,5000.0,16x7,4.0,112,1.0,100,0.0,0.0,"np.arange(-0.34, 0.6, 0.06)","np.arange(5.76, 6.12, 0.06)"


In [19]:
for col in cols:
    print(f'{col}', '\t', (elog_entries[col].values[0]))

Sample 	 ferritin_conc_gly_50_6
comment 	 xpcs, 4 reps per spot, 220K, full beam
Short comment 	 nan
Sample no. (label) 	 10.6
measurement folder 	 ferritin_conc_gly_50_6_0005
scan number 	 1-460
LN flow meter 	 ~2
Temperature, K 	 220.0
Temperature, C 	 nan
Exposure time (s) 	 0.0002
number of frames 	 5000.0
Mesh (spots x lines) 	 16x7
reps per spot 	 4.0
No of spots 	 112
Total exposure time (s) 	 1.0
transmission (%) 	 100
Absorbers 80um 	 0.0
Absorbers 100um 	 0.0
position y, mm 	 np.arange(-0.34, 0.6, 0.06)
position z, mm 	 np.arange(5.76, 6.12, 0.06)


In [29]:
with h5py.File("test.h5", 'a') as f: # w for read/write/create access
    for col in cols:
        f[f'/elog/{col}'] = elog_entries[col].values[0]

if os.path.isfile("test.h5"):
    print("true")

true


In [30]:
# -- visit function for h5 viewing the structure of files
def visit_func(name, node):
    '''
    Return all groups and datasets name and shapes of h5 file called name
    '''
    if isinstance(node, h5py.Group):
        print("group", node.name)
    elif isinstance(node, h5py.Dataset):
        if (node.dtype == 'object') :
            print (node.name, 'is an object Dataset')
        else:
            print('\t', node.name, node.shape)
    else:
        print(node.name, 'is an unknown type')

In [33]:
with h5py.File("test.h5", 'a') as f:
    f.visititems(visit_func)

group /elog
	 /elog/Absorbers 100um ()
	 /elog/Absorbers 80um ()
	 /elog/Exposure time (s) ()
/elog/LN flow meter is an object Dataset
/elog/Mesh (spots x lines) is an object Dataset
/elog/No of spots is an object Dataset
/elog/Sample is an object Dataset
/elog/Sample no. (label) is an object Dataset
	 /elog/Short comment ()
	 /elog/Temperature, C ()
	 /elog/Temperature, K ()
	 /elog/Total exposure time (s) ()
/elog/comment is an object Dataset
/elog/measurement folder is an object Dataset
	 /elog/number of frames ()
/elog/position y, mm is an object Dataset
/elog/position z, mm is an object Dataset
	 /elog/reps per spot ()
/elog/scan number is an object Dataset
/elog/transmission (%) is an object Dataset


In [25]:
if os.path.isfile("test.h5"):
    print("true")
    os.system('rm -f test.h5')

In [21]:
!ls

01-mask-setup.ipynb  elog.ipynb  filtering.ipynb  interpretation.ipynb	test.h5


In [9]:
# elog[(elog['Sample'].str.contains('ferritin_conc120_gly_50_1', na=False))]

In [11]:
elog[elog['Sample'].str.contains('ferritin_conc120_gly_50_1', na=False) & (elog['Temperature, K']==240)]

,Sample,comment,Short comment,Sample no. (label),measurement folder,scan number,LN flow meter,"Temperature, K","Temperature, C",Exposure time (s),number of frames,Mesh (spots x lines),reps per spot,No of spots,Total exposure time (s),transmission (%),Absorbers 80um,Absorbers 100um,"position y, mm","position z, mm"
196,ferritin_conc120_gly_50_1,xpcs_mesh 4 reps,died after 30th position,121,ferritin_conc120_gly_50_1_0006,6,NaN,240.0,NaN,0.0002,5000.0,NaN,4.0,NaN,1.0,100,0.0,0.0,NaN,NaN
197,ferritin_conc120_gly_50_1,xpcs_mesh 4 reps,NaN,121,ferritin_conc120_gly_50_1_0007,7,NaN,240.0,NaN,0.0002,5000.0,NaN,4.0,NaN,1.0,100,0.0,0.0,NaN,NaN
